In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# Aluna: Lídia Medeiros de Lucena Simões
# Turma: Pós-Graduação em Ciências de Dados - IESB - Asa Sul
# Matéria: Data Mining e Machine Learning II

# **Trabalho Final - DMML II
# **


**Introdução

O presente trabalho de conclusão da disciplina Data Mining e Machine Learning II utilizou a base HMEQ_data que traz informações acerca de Empréstimo.

A base traz 5.960 informações referentes a operações de crédito(empréstimo), sendo composta por 13 colunas. A variável de resposta - BAD - é binária e indica quando o cliente é um mau pagador/não pagou o empréstimo (1) ou é um bom pagador/pagou o empréstimo (0). 

A necessidade de se analisar esta base surge da identificação de um alto índice de inadimplência junto às instituições financeiras e a possibilidade de se criar um modelo para prever se um cliente irá honrar ou não a sua dívida/empréstimo. 

Inicialmente, vou criar o dataframe e chamá-lo de EM.

In [ ]:
EM = pd.read_csv('/kaggle/input/hmeq-data/hmeq.csv')

EM.head(5)

Para dar continuidade, preciso saber as informações que constam no arquivo e o seu tamanho:

In [ ]:
print ('EM:' , EM.shape)

In [ ]:
EM.info()

As variáveis constantes na base são:

BAD: 1 = client defaulted on loan 0 = loan repaid
LOAN: Amount of the loan request
MORTDUE: Amount due on existing mortgage
VALUE: Value of current property
REASON: DebtCon = debt consolidation HomeImp = home improvement
JOB: Six occupational categories
YOJ: Years at present job
DEROG: Number of major derogatory reports
DELINQ: Number of delinquent credit lines
CLAGE: Age of oldest trade line in months
NINQ: Number of recent credit lines
CLNO: Number of credit lines
DEBTINC: Debt-to-income ratio


Outra forma de visualizar os dados:

In [ ]:
EM.sample(10).T

Como já pode ser observado, vários dados na tabela estão em branco/nulos. Vou identificar quais valores são nulos e excluí-los da base:

In [ ]:
EM.isnull().sum()

In [ ]:
EM.dropna(axis=0, how='any', inplace=True)
EM.info()

In [ ]:
#Para melhorar a análise da minha base, vou criar uma variável para demonstrar o número de linhas de crédito recente (NINQ) menos o número de linhas de crédito inadimplentes (DELINQ)

EM['NINQ_DELINQ'] = EM['NINQ'] - EM['DELINQ']
EM.info()

**Análise das variáveis

Agora, vou demonstrar em gráficos a frequência das variáveis da base:

In [ ]:
#Importando as bibliotecas

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
EM.hist(figsize=(20,10))

Especialmente em relação à variável BAD, temos o seguinte gráfico:

In [ ]:
ax = sns.countplot(y=x, data=EM).set_title('BAD')

A partir deste gráfico, conseguimos concluir que a maioria dos clientes estão cumprindo com as suas obrigações e pagando os empréstimos, uma vez que o 0 significa adimplência e 1 inadimplência. 

In [ ]:
#Separando as colunas para usar no treino

EM_feats = [c for c in EM.columns if c not in ['NINQ_DELINQ']]
EM_feats = EM[EM_feats]

In [ ]:
plt.figure(figsize=(18,18))
c = 1
for i in EM_feats.columns:
    if c < len(EM_feats.columns):
        plt.subplot(3,3,c)
        sns.boxplot(x='NINQ_DELINQ' , y= i, data=EM)
        c+=1
    else:
        sns.boxplot(x='NINQ_DELINQ' , y= i, data=EM)
plt.tight_layout() 

In [ ]:
#Visualizando o gráfico da variável criada - 'NINQ_DELINQ'
    
sns.set( rc = {'figure.figsize': (10, 10)})
EM_feats = ['NINQ_DELINQ']

for col in EM_feats:
    plt.figure()
    sns.countplot(x=EM[col], data=EM, palette="Set3")
    plt.show()

**Análise com base no Modelo RandomForest

Primeiro, vou importar as bibliotecas:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

%matplotlib inline
import seaborn as sns

Primeiro, vou criar a base de Treino e de Teste

In [ ]:
train, test = train_test_split(EM, random_state=42)

train.shape, test.shape

In [ ]:
#Agora identificar a base de Treino com a variável 'NINQ_DELINQ'

train['NINQ_DELINQ'].value_counts(normalize=True)

In [ ]:
#Agora identificar a base de Teste com a variável 'NINQ_DELINQ'

test['NINQ_DELINQ'].value_counts(normalize=True)

In [ ]:
# Instanciando o RandomForest Classifier

RF = RandomForestClassifier(n_jobs=-1, oob_score=True, n_estimators=200, random_state=42)

In [ ]:
#Treinando o Modelo e Identificando a acurácia do modelo

RF.fit(train[EM_feats], train['NINQ_DELINQ'])
accuracy_score(test['NINQ_DELINQ'], RF.predict(test[EM_feats]))

In [ ]:
#Fazendo as previsões usando o Modelo com a variável 'NINQ_DELINQ'

test['NINQ_DELINQ'] = RF.predict(test[EM_feats]).astype(int)

In [ ]:
#Verificando os resultados das previsões - TESTE

test['NINQ_DELINQ'].value_counts(normalize=True)

In [ ]:
#Verificando os resultados das previsões - TREINO

train['NINQ_DELINQ'].value_counts(normalize=True)

**Conclusão

Após a aplicação da base de teste e treino, cheguei a um Nível de Acurácia de 99%.